In [1]:
#Importing importing Files

import re
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_lg")

In [2]:
#Reading File Data

fileData=[]
import os

dest_folder = "NLP_ModelData_v13"
input_file_loc = os.path.join(dest_folder, "Dataset_DataGenerator.txt")

with open(input_file_loc, "r") as file:
    for line in file:
        x = re.sub("=", " = ", line)
        x = re.sub("  "," ", x)   
        fileData.append(x)

In [3]:
#Loading Shelf Data from "Data Generator"
import shelve
import os

dest_file = os.path.join(dest_folder, "Dict_Lists.shlf")

shelf = shelve.open(dest_file)

dataType_number = shelf["dataType_number"]
dataType_char = shelf["dataType_char"]
dataType_list = shelf["dataType_list"]
dataType_bool = shelf["dataType_bool"]
dataType_class = shelf["dataType_class"]
dataType_void = shelf["dataType_void"]
dataType = shelf["dataType"]

variables_list = shelf["variables_list"]
func_name = shelf["func_name"]

scope_of_variable = shelf["scope_of_variable"]

operators = shelf["operators"]

command_declare = shelf["command_declare"]
command_print = shelf["command_print"]
command_input = shelf["command_input"]

func_prog = shelf["func_prog"]

special_class = shelf["special_class"]
special_function = shelf["special_function"]

other_tags = shelf["other_tags"]
seperators = shelf["seperators"] 
equal_symbol = shelf["equal_symbol"]


shelf.close()

#print(dataType_number, "\n")
#print(dataType, "\n")
#print(variables_list, "\n")
#print(scope_of_variable, "\n")
#print(operators, "\n")
#print(command_declare, "\n")
#print(command_print, "\n")
#print(command_input, "\n")
#print(func_prog, "\n")
#print(special_class, "\n")
#print(special_function, "\n")

In [4]:
#Defining Patterns
#{"IN":["","s"]}
matcher = Matcher(nlp.vocab)

pattern1_1 = [{"LOWER":{"IN":dataType}}]
matcher.add("DATATYPE", None, pattern1_1)

pattern2_1 = [{"LOWER":"global"}]
pattern2_2 = [{"LOWER":"local"}]
matcher.add("SCOPE", None, pattern2_1, pattern2_2)

pattern3_1 = [{"LOWER":{"IN":operators}}]
pattern3_2 = [{"LOWER":{"IN":["additions", "adds"]}}]
pattern3_3 = [{"LOWER":{"IN":["subtractions", "subs"]}}]
pattern3_4 = [{"LOWER":{"IN":["minuses", "multiplies", "muls"]}}]
pattern3_5 = [{"LOWER":{"IN":["divides", "divs"]}}]
pattern3_6 = [{"LOWER":{"IN":["modulus", "moduluses", "mods"]}}]
matcher.add("OPERATOR", None, pattern3_1, pattern3_2, pattern3_3, pattern3_4, pattern3_5, pattern3_6)

pattern4_1 = [{"LOWER":{"IN":command_declare}}]
matcher.add("INIT_COMMAND", None, pattern4_1)

pattern5_1 = [{"LOWER":{"IN":command_print}}]
pattern5_2 = [{"LOWER":{"IN":command_input}}]
pattern5_3 = [{"LOWER":{"IN":special_function}}]
matcher.add("COMMAND", None, pattern5_1, pattern5_2, pattern5_3)

pattern6_1 = [{"LOWER":{"IN":func_prog}}]
pattern6_2 = [{"LOWER":{"IN":["funcs","functions"]}}]
pattern6_3 = [{"LOWER":{"IN":["programs","programmes","progs", "prgs"]}}]
matcher.add("FUNC_PROG", None, pattern6_1, pattern6_2, pattern6_3)

pattern7_1 = [{"LOWER": {"IN": variables_list}}]
matcher.add("VARI", None, pattern7_1)

pattern8_1 = [{"LOWER": {"IN": special_class}}]
matcher.add("SPL_CLASS", None, pattern8_1)

pattern9_1 = [{"LIKE_NUM": True}]
matcher.add("VALUE", None, pattern9_1)

pattern10_1 = [{"LOWER": {"IN": seperators}}]
matcher.add("SEPERATOR", None, pattern10_1)

#pattern10_1 = [{"LOWER":{"IN":equal_symbol}}, {"ORTH": "to", "OP":"?"}]
#pattern10_1 = [{"TEXT": {"REGEX": u"[Ee][Qq][Uu][Aa][Ll][Ss]?( [Tt][Oo])?"}}]
#pattern10_2 = [{"TEXT": {"REGEX": "[Ee][Qq][Uu][Aa][Ll][Ss]?"}}, {"ORTH": {"REGEX": " [Tt][Oo]"}, "OP":"?" }]
#pattern10_2 = [{"LOWER":"equals"}, {"TEXT": {"REGEX": "( [Tt][Oo])"}, "OP":"?"}]
#pattern10_3 = [{"LOWER":"equal"}, {"TEXT": {"REGEX": "( [Tt][Oo])"}, "OP":"?"}]
#matcher.add("EQUAL", None, pattern10_1, pattern10_2)#, pattern10_3)#, pattern10_4, pattern10_5)
pattern10_1 = [{"ORTH":"="}]
matcher.add("EQUAL", None, pattern10_1)

pattern11_1 = [{"ORTH": "'"}, {"TEXT": {"REGEX": "."}, "OP":"*"},{"ORTH": "'"}]
pattern11_2 = [{"ORTH": "\""}, {"TEXT": {"REGEX": "."}, "OP":"*"},{"ORTH": "\""}]
matcher.add("TEXT", None, pattern11_1, pattern11_2)

pattern12_1 = [{"LOWER": {"IN": func_name}}]
matcher.add("FUNC_NAME", None, pattern12_1)

#pattern12_1 = [{"ORTH": "two"}]
#pattern12_2 = [{"ORTH": "3"}]
#matcher.add("COUNT", None, pattern12_1, pattern12_2)

#pattern12_1 = [{"LOWER": {"IN": other_tags}}]
#matcher.add("OTHER", None, pattern12_1)

In [5]:
#Creating Training Data

TRAINING_DATA = []

# Create a Doc object for each text in TEXTS
for doc in nlp.pipe(fileData):
    
    #x = re.sub("=", " = ", doc)
    #doc = re.sub("  "," ", x)
    
    '''
    #Regex Matcher - equals to
    expression = r"[Ee][Qq][Uu][Aa][Ll][Ss]?( [Tt][Oo])?"
    for match in re.finditer(expression, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end)
        span.append(doc.char_span(start, end))
        entity.append("EQUAL")
        # This is a Span object or None if match doesn't map to valid token sequence
        #if span is not None:
        #    print("Found match:", span.text, span.start_char, span.end_char)
    '''
     
    # Match on the doc and create a list of matched spans
    spans = [doc[start:end] for match_id, start, end in matcher(doc)]
    #print(spans)
    
    #Modified Start
    entity_label=[]
    entity_label = [(nlp.vocab.strings[match_id]) for match_id, start, end in matcher(doc)]
    #print(entity_label)
    
    #Regex Matcher - equals to
    expression = r"[Ee][Qq][Uu][Aa][Ll][Ss]?( [Tt][Oo])?"
    for match in re.finditer(expression, doc.text):
        start, end = match.span()
        #span = doc.char_span(start, end)
        spans.append(doc.char_span(start, end))
        entity_label.append("EQUAL")
    

    entities = [(span.start_char, span.end_char, entity_label[index]) for index,span in enumerate(spans)]
    #print(entities)
    #tags = [(entity_label[index]) for index,span in enumerate(spans)]
    
    #for eachWord in entities:
    #    print("Entities: {}".format(eachWord))
    
    # Format the matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    #training_example = (doc.text, {"entities": entities}, {"tags": tags})
    
    # Append the example to the training data
    TRAINING_DATA.append(training_example)
    #Modified End
    
    '''
    entities = [(span.start_char, span.end_char, "INITIALIZATION") for span in spans]
    
    for eachWord in entities:
        print("Entities: {}".format(eachWord))
        
    # Format the matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    # Append the example to the training data
    TRAINING_DATA.append(training_example)
    '''
    

#print(*TRAINING_DATA, sep="\n")

In [6]:
#Store to Data file for inspection

train_data_loc = os.path.join(dest_folder, "training_data.txt")
with open(train_data_loc,"w") as o_file:
    for item in TRAINING_DATA:
        o_file.write("{} \n".format(item))

In [7]:
#Store Pickle Data
import pickle

train_pickle_loc = os.path.join(dest_folder, "training_data.pkl")

with open(train_pickle_loc, 'wb') as f:
    pickle.dump(TRAINING_DATA, f)

In [8]:
#Load Pickle data
with open(train_pickle_loc, 'rb') as f:
    TRAINING_DATA = pickle.load(f)

In [10]:
#Training Loop
import random

nlp = spacy.blank("en")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)

#tagger = nlp.create_pipe("tagger")
#nlp.add_pipe(tagger) 


ner.add_label("DATATYPE")
ner.add_label("SCOPE")
ner.add_label("OPERATOR")
ner.add_label("INIT_COMMAND")
ner.add_label("COMMAND")
ner.add_label("FUNC_PROG")
ner.add_label("FUNC_NAME")
ner.add_label("VARI")
ner.add_label("SPL_CLASS")
ner.add_label("VALUE")
ner.add_label("COUNT")
ner.add_label("SEPERATOR")
ner.add_label("EQUAL")
ner.add_label("TEXT")
#ner.add_label("OTHER")

'''
tagger.add_label("DATATYPE")
tagger.add_label("SCOPE")
tagger.add_label("OPERATOR")
tagger.add_label("COMMAND")
tagger.add_label("FUNC_PROG")
tagger.add_label("FUNC_NAME")
tagger.add_label("VARI")
tagger.add_label("SPL_CLASS")
tagger.add_label("VALUE")
tagger.add_label("COUNT")
tagger.add_label("SEPERATOR")
tagger.add_label("EQUAL")
tagger.add_label("TEXT")
tagger.add_label("OTHER")
'''

'''
optimizer = SGD(
    learn_rate=0.001,
    L2=1e-6,
    grad_clip=1.0
)
'''

print(nlp.pipe_names)

import time
start_time = time.time()

# Start the training
nlp.begin_training()

# Loop for 10 iterations

optimizer = ner.create_optimizer()
optimizer.learn_rate = 0.001
optimizer.beta1=0.9
optimizer.beta2=0.999
optimizer.eps=1e-8
optimizer.L2=1e-6
optimizer.max_grad_norm=1.0
batch_size = 5000
dropout_rate = 0.1
total_epoch=5

#Printing Start
print("Optimizer: \t", "SGD")
print("Learn Rate: \t", optimizer.learn_rate)
print("Beta1: \t\t", optimizer.beta1)
print("Beta2: \t\t", optimizer.beta2)
print("Epsilon: \t", optimizer.eps)
print("L2: \t\t", optimizer.L2)
print("Max Grad Norm: \t", optimizer.max_grad_norm)
print("Batch Size: \t", batch_size)
print("Droupout Rate: \t", dropout_rate)
print("Total Epoch: \t", total_epoch)

#Printing End

for itn in range(total_epoch):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=batch_size):
        texts = [text for text, entities in batch]
        annotations = [entities for text, entities in batch]
        #for text, entities, tags in batch:
        #    annotations.append(entities)   
        #    annotations.append(tags)
        
        #print(texts)
        #print(annotations)

        # Update the model
        nlp.update(texts, annotations, losses=losses, drop=dropout_rate, sgd=optimizer)
               
        #annotations_tag = [tags for text, entities, tags in batch]
        #print(annotations_tag)
        #nlp.update(texts, annotations_tag, losses=losses)
    print("====================================")    
    print("Epoch:{}/{}".format(itn+1, total_epoch))
    print("Losses: "+ str(losses))
    
    end_time = time.time()
    print("Time Since Start:", end_time - start_time)
    

['ner']
Optimizer: 	 SGD
Learn Rate: 	 0.001
Beta1: 		 0.9
Beta2: 		 0.999
Epsilon: 	 1e-08
L2: 		 1e-06
Max Grad Norm: 	 1.0
Batch Size: 	 5000
Droupout Rate: 	 0.1
Total Epoch: 	 5
Epoch:1/5
Losses: {'ner': 611403.1619873047}
Time Since Start: 99.1434006690979
Epoch:2/5
Losses: {'ner': 200406.1436767578}
Time Since Start: 199.16060137748718
Epoch:3/5
Losses: {'ner': 9531.2309268713}
Time Since Start: 300.4048020839691
Epoch:4/5
Losses: {'ner': 481.8201700129721}
Time Since Start: 405.4708025455475
Epoch:5/5
Losses: {'ner': 7.4357797083963675}
Time Since Start: 511.4210035800934


In [11]:
#Saving the nlp Object
nlp.to_disk("./NLP_Object")

In [12]:
#Testing the models
#https://stackoverflow.com/questions/44827930/evaluation-in-a-spacy-ner-model
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

# example run

examples = [
    ("Write a function to creates global integer",
     [(0, 5, 'COMMAND'), (8, 16, 'FUNC_PROG'), (20, 27, 'INIT_COMMAND'), (28, 34, 'SCOPE'), (35, 42, 'DATATYPE')]),
    ("Write function to sets global strs strs1 = 'please enter value'",
     [(0, 5, 'COMMAND'), (6, 14, 'FUNC_PROG'), (18, 22, 'INIT_COMMAND'), (23, 29, 'SCOPE'), (30, 34, 'DATATYPE'), (35, 40, 'VARI'), (41, 42, 'EQUAL'), (43, 63, 'TEXT')])
]

ner_model = spacy.load("./NLP_Object") # for spaCy's pretrained use 'en_core_web_sm'
results = evaluate(ner_model, examples)
print(results)

{'uas': 0.0, 'las': 0.0, 'las_per_type': {'': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'ents_p': 100.0, 'ents_r': 100.0, 'ents_f': 100.0, 'ents_per_type': {'COMMAND': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'SCOPE': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'INIT_COMMAND': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'DATATYPE': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'FUNC_PROG': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'TEXT': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'EQUAL': {'p': 100.0, 'r': 100.0, 'f': 100.0}, 'VARI': {'p': 100.0, 'r': 100.0, 'f': 100.0}}, 'tags_acc': 0.0, 'token_acc': 100.0, 'textcat_score': 0.0, 'textcats_per_cat': {}}


In [23]:
#NER Test
from spacy import displacy

#doc = ("create a function to add numbers z and b")
#doc = ("if a > b then add a and b")
doc = ("get input k")
#doc = ("if a>b then print a. If b>c then print c")

#doc = ("display variable text1 equals to 'Wubba lubba dubdub! General Kenobi A sentence' on screen")
#doc = ("Create a program and name it wazza_wazza for create local integer equal to 45")

#doc = re.sub("=", " = ", doc)
#doc = re.sub("  "," ", doc) 

doc = nlp(doc)
  

for ent in doc.ents:
    print (ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(nlp(doc.text), style="ent", jupyter=True)

get 0 3 COMMAND
input 4 9 COMMAND
k 10 11 VARI


In [21]:
# Tagger Test

test_text = "Write a program for mod doubles b = 77 and c = doubles 77"
doc = nlp(test_text)
print("Tags", [(t.text, t.tag_, t.pos_) for t in doc])

Tags [('Write', '', ''), ('a', '', ''), ('program', '', ''), ('for', '', ''), ('mod', '', ''), ('doubles', '', ''), ('b', '', ''), ('=', '', ''), ('77', '', ''), ('and', '', ''), ('c', '', ''), ('=', '', ''), ('doubles', '', ''), ('77', '', '')]


In [ ]:
#Importing the nlp Object
import spacy
from spacy import displacy
nlp = spacy.load("./NLP_Object")

doc = nlp("create a function to add numbers a and b")
for ent in doc.ents:
    print (ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(nlp(doc.text), style="ent", jupyter=True)